<a href="https://colab.research.google.com/github/mille055/duke_chatbot/blob/main/notebooks/Copy_of_Create_ft_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pinecone-client
!pip install sentence_transformers
!pip install openai
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [34]:
import argparse
import hashlib
import io, re, os
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, PipelineTool
import openai
from pinecone import init, Index, Pinecone, ServerlessSpec
import pinecone
import logging

In [6]:
!git clone https://github.com/mille055/duke_chatbot.git

Cloning into 'duke_chatbot'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 155 (delta 58), reused 64 (delta 38), pack-reused 61
Receiving objects: 100% (155/155), 17.18 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [9]:
qa_pairs_json = json.load(open('/content/duke_chatbot/data/fine_tune_data.json'))


In [61]:
qa_pairs_json[0]

{'input': "What does Duke's Master of Engineering in AI for Product Innovation aim to develop?",
 'output': "Duke's Master of Engineering in AI for Product Innovation aims to develop technical leaders who are equipped to build our future through AI and Machine Learning."}

In [55]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('pinecone_api_key2')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


In [56]:
pc = Pinecone(api_key=PINECONE_API_KEY)


# List all indexes in your project
print(pc.list_indexes())


{'indexes': [{'dimension': 384,
              'host': 'newindex-1r32hfx.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'newindex',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [57]:
def semantic_search(query, top_k = 3):
  embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
  model = SentenceTransformer(embedding_model)
  index_name = 'newindex'
  pc = Pinecone(api_key=PINECONE_API_KEY)
  index = pc.Index(index_name)
  texts = []
  source_list = []
  try:
    query_embedding = model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    matches = [match for match in results["matches"]]
    for match in matches:
        source_list.append(match['metadata']['source'])
        texts.append(match['metadata']['text'])
    return texts, source_list
  except Exception as e:
    print(f"Error during semantic search: {e}")
    return [], []

In [60]:
test = semantic_search("how do i apply to the AIPI program?")
print(test)

(['the Certificate program. An application for the AI Foundations for Product Innovation graduate certificate program requires the following: A bachelor’s degree in engineering or science from an accredited institution (transcripts required, including an estimated GPA and a grade scale) Statement of purpose Résumé Two recommendations English Language Testing (TOEFL or IELTS): official results required— international applicants only Video introduction This is a standalone certificate program and does not qualify international students for US visa sponsorship. ', 'ineering degree program. courses Semester Course Titles Summer (Pre-Program) Python & Data Science Math Boot Camp Fall 1 AIPI 510: Sourcing Data for Analytics Spring 1 AIPI 520: Modeling Process & Algorithms Summer 1 AIPI 540: Building Products Using Deep Learning Fall 2 Departmental Technical Elective Students who complete the certificate earn credits they can apply toward the online Duke AI Master of Engineering degree. ', 'o

In [62]:
# prompt: I have a json file with 'input' and 'output' and want to add the result of semantic_search(input) as well as change the names of 'input' and 'output' to 'query' and 'answer'

for qa_pair in qa_pairs_json:
  query, _ = semantic_search(qa_pair['input'])
  qa_pair['context'] = query
  qa_pair['query'] = qa_pair['input']
  qa_pair['answer'] = qa_pair['output']
  # del qa_pair['input']
  # del qa_pair['output']

with open('updated_qa_pairs.json', 'w') as f:
  json.dump(qa_pairs_json, f)


In [63]:

from google.colab import files

files.download('updated_qa_pairs.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
updated_qa_pairs.json